In [ ]:
using Base.Meta

In [ ]:
a_expr = Meta.parse("(b & !c) | (!b & c)")

In [ ]:
show_sexpr(a_expr)

In [1]:
# The simple building block of our ROBDDs.
struct BDDNode
    var::Int64
    lo::Int64
    hi::Int64
end

In [6]:
# We define a `ROBDDContext` that 
# (1) tracks the variables we're working with, and their ordering; and
# (2) maintains some hash tables for bookkeeping

mutable struct NodeTable
    d::Dict{Int64,BDDNode}
    latest::Int64
end

function NodeTable(n::Int64)
    d = Dict{Int64,BDDNode}()
    d[0] = BDDNode(n+1,n+1,n+1)
    d[1] = BDDNode(n+1,n+1,n+1)
    return NodeTable(d,1)
end

mutable struct LookupTable
    d::Dict{BDDNode,Int64}
end

LookupTable() = LookupTable(Dict{BDDNode,Int64}())

struct ROBDDContext
    order::Vector{Symbol}
    T::NodeTable
    H::LookupTable
end

ROBDDContext(order::Vector{Symbol}) = ROBDDContext(order, NodeTable(length(order)), LookupTable())

ROBDDContext

In [3]:
# Some essential functionality for the ROBDDContext 
# and its members.

import Base: getindex, setindex!

# Basic operations for the node table
function add!(T::NodeTable, node::BDDNode)
    T.latest = T.latest + 1
    T.d[T.latest] = node
    return T.latest
end

getindex(T::NodeTable, idx::Int64) = T.d[idx]


# Basic operations for the "inverse" lookup table
member(H::LookupTable, node::BDDNode) = haskey(H.d, node)

getindex(H::LookupTable, node::BDDNode) = H.d[node]

function setindex!(H::LookupTable, idx::Int64, node::BDDNode)
    setindex!(H.d, idx, node)
end

function get_or_add(ctxt::ROBDDContext, node::BDDNode)
    if node.lo == node.hi
        return node.lo
    elseif member(ctxt.H, node)
        return ctxt.H[node]
    else
        u = add!(ctxt.T, node)
        ctxt.H[node] = u
        return u
    end
end

var(ctxt::ROBDDContext, idx::Int64) = ctxt.T[idx].var
lo(ctxt::ROBDDContext, idx::Int64) = ctxt.T[idx].lo
hi(ctxt::ROBDDContext, idx::Int64) = ctxt.T[idx].hi

hi (generic function with 1 method)

In [9]:
function apply(ctxt::ROBDDContext, op::Function, idx1::Int64, idx2::Int64; 
               memo::Dict{Tuple{Function,Int64,Int64},Int64}=Dict{Tuple{Function,Int64,Int64},Int64}())

    function rec_app(u1, u2)
        if haskey(memo, (op,u1,u2))
            return memo[(op,u1,u2)]
        elseif in(u1, (0,1)) & in(u2, (0,1))
            u = Int64(op(Bool(u1), Bool(u2)))
        elseif var(ctxt, u1) == var(ctxt, u2)
            u = get_or_add(var(ctxt,u1), 
                           rec_app(lo(ctxt,u1), lo(ctxt,u2)), 
                           rec_app(hi(ctxt,u1), hi(ctxt,u2)))
        elseif var(ctxt, u1) < var(ctxt, u2)
            u = get_or_add(var(ctxt,u1),
                           rec_app(lo(ctxt, u1), u2),
                           rec_app(hi(ctxt, u1), u2)
                           )
        else
            u = get_or_add(var(ctxt,u2),
                           rec_app(u1, lo(ctxt, u2)),
                           rec_app(u1, hi(ctxt, u2))
                           )
        end
        memo[(op,u1,u2)] = u
        return u
    end
    
    return rec_app(idx1, idx2)
end

apply (generic function with 1 method)

In [10]:
my_ctxt = ROBDDContext([:x1,:x2])

ROBDDContext([:x1, :x2], NodeTable(Dict{Int64,BDDNode}(0 => BDDNode(3, 3, 3),1 => BDDNode(3, 3, 3)), 1), LookupTable(Dict{BDDNode,Int64}()))

In [13]:
apply(my_ctxt, |, 0, 0)

1

In [ ]:
myexpr = Meta.parse("a | b | c | d | e")

In [ ]:
myexpr = 

In [ ]:
Expr(:call, :&, :f, a)

In [ ]:
myexpr = Expr(:call, :&, true, false)

In [ ]:
supertype(Symbol)

In [ ]:
function dumb_func(foo::Bool, bar::Bool)::Bool
    return false
end

In [ ]:
println(typeof(dumb_func))

In [ ]:
fing = methods(dumb_func)

In [ ]:
typeof(&) <: Function

In [ ]:
function dumb_func(x)
    function sub_func(y)
        return x + y
    end
    
    return sub_func(x)
end

In [ ]:
dumb_func(10)